In [1]:
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib.animation as animation
# from celluloid import Camera
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from pathlib import Path
import os
import imageio
import itertools
from scipy import fft
from conda_env import env

%matplotlib

Using matplotlib backend: Qt5Agg


Using matplotlib backend: Qt5Agg


In [5]:

def Crank(Nx,Ny,Nz,f,X=5,Y=5):
        Nx = Nx
        X = X;
        dx = X/Nx;
        x = np.linspace(-X/2,X/2,Nx,endpoint=True)
        f = f'{f:0.2f}'
        # hi = 1

        Ny = Ny;
        Y = Y;
        dy = Y/Ny;
        y = np.linspace(-Y/2,Y/2,Ny,endpoint=True)



        nxy , nyx = np.meshgrid(y,x)
        nxy , nyx = nyx, nxy

        E0 =np.zeros([Nx,Ny],dtype=complex)
        for ind in range(len(x_m)):
            # x0 , y0 = np.meshgrid(x - x[ind], y - y[ind])
            E_ = np.exp(-((nxy-x_m[ind])**2 + (nyx-y_m[ind])**2)/2)
            E0 += np.copy(E_)



        # print(f'f={f}')

        z = 0
        Nz = Nz
        zm = np.linspace(0,2*float(f),Nz,endpoint=False)
        dz = zm[1]

        E_lens = np.exp(1j*(nxy**2 + nyx**2)/(2*float(f)))
        E_s = E0 * E_lens



        Ax = np.zeros(Nx) + 1.0
        Cx = np.zeros(Nx) + 1.0

        Cx[0] = 0.
        Ax[-1] = 0.


        Ay = np.zeros(Ny) + 1.0
        Cy = np.zeros(Ny) + 1.0

        Cy[0] = 0
        Ay[-1] = 0


        E = np.copy(E_s)


        Bx = np.zeros(Nx,dtype=complex) -2.0 * (1 + dx**2/dz/hi)
        Bx[0] = 1.
        Bx[-1] = 1.

        By = np.zeros(Ny,dtype=complex) -2.0 * (1 + dy**2/dz/hi)
        By[0] = 1
        By[-1] = 1



        direction01 = f'./2.1/'
        direction1 =f'/{Nx}x{Ny}/'
        direction2 = f'f={f}/'
        direction31 = f'full'
        direction = f'{direction01}{direction1}{direction2}'

        Path(f"{direction01}{direction1}{direction2}").mkdir(parents=True, exist_ok=True)
        [os.remove(f'{direction01}{direction1}{direction2}{file}') for file in os.listdir(f'{direction01}{direction1}{direction2}') if file.endswith('.png') or file.endswith('.gif')]


        Ean = 1/(1 - z/float(f) - 1j*z) * np.exp(-(nxy**2 + nyx**2)/2 *(1+1j*(z - 1/float(f) *(1-z/float(f))))/((1-z/float(f))**2+z**2));
        Eanprint = np.abs(Ean)**2;

        Ep = np.abs(E_s)**2;

        fig1 = plt.figure(1,figsize=(9.5,10));
        fig1.clf()
        ax11 = fig1.add_subplot(2,2,1,projection='3d')
        ax11.plot_surface(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
        plt.xlabel('X')
        plt.ylabel('Y')
        # plt.colorbar();
        # axis square;
        plt.title(f'Analytic with z={z:0.2f}\n f={f}');

        ax12 = fig1.add_subplot(2,2,3)
        ims11 = ax12.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
        ax12.set_aspect('equal', adjustable='box')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.colorbar(ims11);
        plt.title(f'Analytic z ={z:0.2f}');
        # axis square;
        # plt.view(2);

        ax13 = fig1.add_subplot(2,2,2,projection='3d')
        ax13.plot_surface(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
        plt.xlabel('X')
        plt.ylabel('Y')
        # plt.colorbar();
        # axis square;
        plt.title(f'z={z:0.2f}\n f={f}');

        ax14 = fig1.add_subplot(2,2,4)
        ims12 = ax14.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
        ax14.set_aspect('equal', adjustable='box')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.colorbar(ims12);

        filename = f'z={z:0.2f}'
        fig1.savefig(f'{direction01}{direction1}{direction2}{filename}.png')
        # plt.view(2);
        # axis square;




        for idz in range(Nz):


            z = z + dz;
            Ean = 1/(1 - z/float(f) - 1j*z) * np.exp(-(nxy**2 + nyx**2)/2 *(1+1j*(z - 1/float(f) *(1-z/float(f))))/((1-z/float(f))**2+z**2));
            Eanprint = abs(Ean)**2;

            # цикл по y с дифракцией по x
            for idy in range(Ny):
            # for idy in [Ny//2]:
                Fx = np.zeros(Nx,dtype=complex);
                alphax = np.zeros(Nx,dtype=complex);
                betax = np.zeros(Nx,dtype=complex);
                for idx in range(1,Nx-1):
                    Fx[idx] = -E[idx-1,idy] + 2 *(1 - dx**2/dz/hi) * E[idx,idy] - E[idx+1,idy];

                Fx[0] = 0;
                Fx[-1] = 0;

                alphax[1] = -Cx[0]/Bx[0];
                betax[1] = Fx[0]/Bx[0];

                # for i in range(2,Nx-1):
                    # alphax[idx+1]= -Cx[idx]/(Ax[idx]*alphax[idx] + Bx[idx]);
                    # betax[idx+1] = alphax[idx+1]/Cx[idx]*(Ax(idx)*betax(idx) - Fx(idx));
                '''решение по теории:'''
                for i in range(1,Nx-1):
                #     alphax[i+1] = -Cx[i]/Bx[i] * (1/(1+Ax[i]/Bx[i]*alphax[i]))
                #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
                    alphax[i+1]= -Cx[i]/(Ax[i]*alphax[i] + Bx[i]);
                    betax[i+1] = alphax[i+1]/Cx[i]*(Ax[i]*betax[i] - Fx[i]);
                #
                '''решение из прошлого семестра'''
                # for i in range(2,Nx):
                #     alphax[i] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i-1])
                #     betax[i] = (Fx[i-1]-Ax[i]*betax[i-1])/(Ax[i]*alphax[i-1]+Bx[i])



                E[-1,idy] = (Fx[-1] - Ax[-1]*betax[-1])/(Ax[-1]*alphax[-1] + Bx[-1]);
                '''по теории'''
                # for idx in range(Nx-1,0,-1):
                #     E[idx-1,idy] = alphax[idx-1] * E[idx,idy] + betax[idx-1]
                '''прошлый семестр'''
                for idx in range(Nx-2 ,-1,-1):
                    # print(idx,idy)
                    E[idx,idy] = alphax[idx+1] * E[idx+1,idy] + betax[idx+1]
            # figure(2);
            # subplot(2,1,1);
            # surf(nxy,nyx,abs(E).^2,linestyle = ':',edgecolor='none');
            # xlabel('X')
            # ylabel('Y')
            # colorbar();
            # axis square;
            # title({['f=' num2str(f)];['z=' num2str(z)]});
            #
            # subplot(2,1,2)
            # surf(nxy,nyx,abs(E),linestyle = ':',edgecolor='none');
            # xlabel('X')
            # ylabel('Y')
            # colorbar();
            # view(2);
            # axis square;
            ''' цикл по x с дифракцией по y '''
            for idx in range(Nx):
            # for idx in [Nx//2]:
                Fy = np.zeros(Ny,dtype=complex);
                alphay = np.zeros(Ny,dtype=complex);
                betay = np.zeros(Ny,dtype=complex);
                for idy in range(2,Ny-1):
                    Fy[idy] = -E[idx,idy-1] + 2 *(1 - dy**2/dz/hi) * E[idx,idy] - E[idx,idy+1];

                Fy[0] = 0;
                Fy[-1] = 0;

                alphay[1] = -Cy[0]/By[0];
                betay[1] = Fy[0]/By[0];

                # for idy in
                #     alphay(idy+1)= -Cy(idy)/(Ay(idy)*alphay(idy) + By(idy));
                #     betay(idy+1) = alphay(idy+1)/Cy(idy)*(Ay(idy)*betay(idy) - Fy(idy));
                '''решение по теории:'''
                for i in range(1,Ny-1):
                #     # alphay[i+1] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i])
                #     # betay[i+1] = (Fy[i-1]-Ay[i]*betay[i])/(Ay[i]*alphay[i]+By[i])
                    alphay[i+1]= -Cy[i]/(Ay[i]*alphay[i] + By[i]);
                    betay[i+1] = alphay[i+1]/Cy[i]*(Ay[i]*betay[i] - Fy[i]);
                '''решение из прошлого семестра'''
                # for i in range(2,Ny):
                #     alphay[i] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i-1])
                #     betay[i] = (Fy[i-1]-Ay[i]*betay[i-1])/(Ay[i]*alphay[i-1]+By[i])

                E[idx,-1] = (Fy[-1] - Ay[-1]*betay[-1])/(Ay[-1]*alphay[-1] + By[-1]);
                '''по теории'''
                # for idy in range(Ny-1,0,-1):
                #     E[idx,idy-1] = alphay[idy-1] * E[idx,idy] + betay[idy-1];
                '''прошлый семестр'''
                for idy in range(Ny-2,-1,-1):
                    E[idx,idy] = alphay[idy+1] * E[idx,idy+1] + betay[idy+1]

                    # for i in range(1,Nx):
                    #     alphax[i+1] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i])
                    #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
                    #
                    #
                    # E[-1,idy] = (Fx[-1] - Ax[-1]*betax[-1])/(Ax[-1]*alphax[-1] + Bx[-1]);
                    # for idx in range(Nx-1,-1,-1):
                    #     E[idx-1,idy] = alphax[idx] * E[idx,idy] + betax[idx]

            Ep = np.abs(E)**2;

            fig2 = plt.figure(2,figsize=(9.5,10));
            fig2.clf()
            ax21 = fig2.add_subplot(2,2,1,projection='3d')
            ax21.plot_surface(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
            plt.xlabel('X')
            plt.ylabel('Y')
            # plt.colorbar();
            # axis square;
            plt.title(f'Analytic with z={z:0.2f}\n f={f}; step={idz+1}/{Nz}');

            ax22 = fig2.add_subplot(2,2,3)
            ims21 = ax22.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
            ax22.set_aspect('equal', adjustable='box')
            plt.xlabel('X')
            plt.ylabel('Y')
            plt.colorbar(ims21);
            plt.title(f'Analytic z ={z:0.2f}');
            # axis square;
            # plt.view(2);

            ax23 = fig2.add_subplot(2,2,2,projection='3d')
            ax23.plot_surface(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
            plt.xlabel('X')
            plt.ylabel('Y')
            # plt.colorbar();
            # axis square;
            plt.title(f'z={z:0.2f}\n f={f}');

            ax24 = fig2.add_subplot(2,2,4)
            ims22 = ax24.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
            ax24.set_aspect('equal', adjustable='box')
            plt.xlabel('X')
            plt.ylabel('Y')
            plt.colorbar(ims22);
            # plt.view(2);
            # axis square;
            filename = f'z={z:0.2f}'
            fig2.savefig(f'{direction01}{direction1}{direction2}{filename}.png')
            plt.pause(0.3)

            if z == float(f):
                fig3 = plt.figure(3,figsize=(12,14))
                fig3.clf()
                ax31 = fig3.add_subplot(3,1,1)
                plt.title(f'Сравнение решений \n z:{z} = f:{f} for x = {x[Nx//2]}')
                point = 1
                ax31.scatter(y[::point],Ep[Nx//2,:][::point],c='r',label='Численной схемой')
                ax31.plot(y,Eanprint[Nx//2,:],label='Аналитически')
                plt.legend()
                plt.xlabel('Y')
                plt.ylabel('I')

                ax32 = fig3.add_subplot(3,2,3)
                plt.title(f'Абсолютная ошибка при x={x[Nx//2]}')
                ax32.stem(y,abs(Eanprint[Nx//2,:]-Ep[Nx//2,:]))
                plt.xlabel('Y')
                plt.ylabel('I')
                filename = f'y=0 in z={f}'

                ax32 = fig3.add_subplot(3,2,4)
                plt.title(f'Абсолютная ошибка при y={y[Ny//2]}')
                ax32.stem(x,abs(Eanprint[:,Ny//2]-Ep[:,Ny//2]))
                plt.xlabel('X')
                plt.ylabel('I')
                filename = f'x=0 in z={f}'

                # fig4 = plt.figure(4,figsize=(9.5,10))
                # fig4.clf()
                ax41 = fig3.add_subplot(3,2,5,projection='3d')
                ax41.plot_surface(nxy,nyx,abs(Eanprint-Ep),linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
                plt.xlabel('X')
                plt.ylabel('Y')
                # plt.colorbar();
                # axis square;
                plt.title(f'Абсолютная ошибка при z:{z}=f:{f}');

                ax42 = fig3.add_subplot(3,2,6)
                ims41 = ax42.pcolormesh(nxy,nyx,abs(Eanprint-Ep),linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
                ax42.set_aspect('equal', adjustable='box')
                plt.xlabel('X')
                plt.ylabel('Y')
                plt.colorbar(ims41);
                plt.title(f'Абсолютная ошибка при z:{z}=f:{f}\n'
                          f'Emax={np.max(abs(Eanprint-Ep)):0.5f}');
                plt.tight_layout()

                Path(f"{direction01}{direction1}f=z").mkdir(parents=True, exist_ok=True)
                fig3.savefig(f'{direction01}{direction1}f=z/{filename}.svg')
                fig3.savefig(f'{direction01}{direction1}f=z/{filename}.png')
                # Path(f"{direction01}{direction1}f=z").mkdir(parents=True, exist_ok=True)
                # filename = f'f=z={z:0.2f}'
                #
                # fig4.savefig(f'{direction01}{direction1}f=z/{filename}.png')

In [6]:
if os.environ['USERDOMAIN'] == 'DESKTOP-RMQ8D6S':
    Nx = 128 + 1
    Ny = 64 + 1
    Nz = 10
    Nf = 10
    N_m = []
    N_m.append((Nx,Ny))
else:
    N = [101,201,301,401,501]
    Nf = 100
    Nz = 100
    N_m = []
    for x1 in itertools.product(N,repeat=2):
        N_m.append(x1)
    print(N_m)


X = 5
Y = 5

f = 0.5;

hi = 1/2j;
x1 , y1 = 0. , -0.
x_m , y_m = [x1] , [y1]


for k in range(len(N_m)):
    Nx, Ny = N_m[k]
    f_m = np.linspace(0,2,Nf,endpoint=False)


    for f in f_m:
        if f == 0:
            continue
        Crank(Nx,Ny,Nz,f,X=5,Y=5)

C:\Users\dimon\AppData\Local\Temp/ipykernel_5884/3975549158.py:92: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims11 = ax12.pcolormesh(nxy,nyx,Eanprint,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
C:\Users\dimon\AppData\Local\Temp/ipykernel_5884/3975549158.py:110: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims12 = ax14.pcolormesh(nxy,nyx,Ep,linestyle = ':',edgecolor='none',cmap=cm.coolwarm);
C:\Users\dimon\AppData\Local\Temp/ipykernel_5884/397554

KeyboardInterrupt: 

In [2]:


if os.environ['USERDOMAIN'] == 'DESKTOP-RMQ8D6S':
    Nx = 512
    Ny = 512
    Nz = 50
    N_m = []
    N_m.append((Nx,Ny))
else:
    N = [101,201,301,401,501]
    Nf = 100
    Nz = 100
    N_m = []
    for x1 in itertools.product(N,repeat=2):
        N_m.append(x1)
    print(N_m)


X = 7
Y = 7

hi = 1/2j;

In [5]:
f = 1
# def Crank2(Nx=Nx,Ny=Ny,Nz=Nz,f=f,X=X,Y=Y):
x1 , y1 = 0. , -0.
x2 , y2 = -2.5 , 1.4
x3 , y3 = -1.5 , -2.1
x4 , y4 = 2.5 , -1.2
x5 , y5 = 1.55 , 2.33

x_m , y_m = [x1,x2,x3,x4,x5] , [y1,y2,y3,y4,y5]

Nx = Nx
X = X;
dx = X / Nx;
x = np.linspace(-X / 2, X / 2, Nx, endpoint=False)
f = f'{f:0.3f}'
# hi = 1

Ny = Ny;
Y = Y;
dy = Y / Ny;
y = np.linspace(-Y / 2, Y / 2, Ny, endpoint=False)

nxy, nyx = np.meshgrid(y, x)
nxy, nyx = nyx, nxy

E0 = np.zeros([Nx, Ny], dtype=complex)
for ind in range(len(x_m)):
    # x0 , y0 = np.meshgrid(x - x[ind], y - y[ind])
    E_ = np.exp(-((nxy - x_m[ind]) ** 2 + (nyx - y_m[ind]) ** 2) / 2)
    E0 += np.copy(E_)

# print(f'f={f}')

z = 0
Nz = Nz
zm = np.linspace(0, float(f), Nz, endpoint=False)
dz = zm[1]

E_lens = np.exp(1j * (nxy ** 2 + nyx ** 2) / (2 * float(f)))
E_s = E0 * E_lens

Ax = np.zeros(Nx) + 1.0
Cx = np.zeros(Nx) + 1.0

Cx[0] = 0.
Ax[-1] = 0.

Ay = np.zeros(Ny) + 1.0
Cy = np.zeros(Ny) + 1.0

Cy[0] = 0
Ay[-1] = 0

E = np.copy(E_s)

Bx = np.zeros(Nx, dtype=complex) - 2.0 * (1 + dx ** 2 / dz / hi)
Bx[0] = 1.
Bx[-1] = 1.

By = np.zeros(Ny, dtype=complex) - 2.0 * (1 + dy ** 2 / dz / hi)
By[0] = 1
By[-1] = 1

direction01 = f'./2.2/'
direction1 = f'/{Nx}x{Ny}/'
direction2 = f'f={f}/'
direction31 = f'full'
direction = f'{direction01}{direction1}{direction2}'

Path(f"{direction01}{direction1}{direction2}").mkdir(parents=True, exist_ok=True)
[os.remove(f'{direction01}{direction1}{direction2}{file}') for file in
 os.listdir(f'{direction01}{direction1}{direction2}') if file.endswith('.png') or file.endswith('.gif')]


# Ep0 = np.abs(fft.fftshift(fft.fft2(fft.fftshift(E_s),workers=7)))**2

# fig1 = plt.figure(1, figsize=(9.5, 10));
#
# ax13 = fig1.add_subplot(2, 2, 1, projection='3d')
# ax13.plot_surface(nxy, nyx, Ep0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
# plt.xlabel('X')
# plt.ylabel('Y')
# # plt.colorbar();
# # axis square;
# plt.title(f'z={z:0.2f}\n f={f}');
#
# ax14 = fig1.add_subplot(2, 2, 3)
# ims12 = ax14.pcolormesh(nxy, nyx, Ep0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
# ax14.set_aspect('equal', adjustable='box')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.colorbar(ims12);

# fig1.show()

# filename = f'z={z:0.2f}'
# fig1.savefig(f'{direction01}{direction1}{direction2}{filename}.png')

[]

In [6]:


for idz in range(Nz):

    z = z + dz;

    # цикл по y с дифракцией по x
    for idy in range(Ny):
        # for idy in [Ny//2]:
        Fx = np.zeros(Nx, dtype=complex);
        alphax = np.zeros(Nx, dtype=complex);
        betax = np.zeros(Nx, dtype=complex);
        for idx in range(1, Nx - 1):
            Fx[idx] = -E[idx - 1, idy] + 2 * (1 - dx ** 2 / dz / hi) * E[idx, idy] - E[idx + 1, idy];

        Fx[0] = 0;
        Fx[-1] = 0;

        alphax[1] = -Cx[0] / Bx[0];
        betax[1] = Fx[0] / Bx[0];

        # for i in range(2,Nx-1):
        # alphax[idx+1]= -Cx[idx]/(Ax[idx]*alphax[idx] + Bx[idx]);
        # betax[idx+1] = alphax[idx+1]/Cx[idx]*(Ax(idx)*betax(idx) - Fx(idx));
        '''решение по теории:'''
        for i in range(1, Nx - 1):
            #     alphax[i+1] = -Cx[i]/Bx[i] * (1/(1+Ax[i]/Bx[i]*alphax[i]))
            #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
            alphax[i + 1] = -Cx[i] / (Ax[i] * alphax[i] + Bx[i]);
            betax[i + 1] = alphax[i + 1] / Cx[i] * (Ax[i] * betax[i] - Fx[i]);
        #
        '''решение из прошлого семестра'''
        # for i in range(2,Nx):
        #     alphax[i] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i-1])
        #     betax[i] = (Fx[i-1]-Ax[i]*betax[i-1])/(Ax[i]*alphax[i-1]+Bx[i])

        E[-1, idy] = (Fx[-1] - Ax[-1] * betax[-1]) / (Ax[-1] * alphax[-1] + Bx[-1]);
        '''по теории'''
        # for idx in range(Nx-1,0,-1):
        #     E[idx-1,idy] = alphax[idx-1] * E[idx,idy] + betax[idx-1]
        '''прошлый семестр'''
        for idx in range(Nx - 2, -1, -1):
            # print(idx,idy)
            E[idx, idy] = alphax[idx + 1] * E[idx + 1, idy] + betax[idx + 1]
    # figure(2);
    # subplot(2,1,1);
    # surf(nxy,nyx,abs(E).^2,linestyle = ':',edgecolor='none');
    # xlabel('X')
    # ylabel('Y')
    # colorbar();
    # axis square;
    # title({['f=' num2str(f)];['z=' num2str(z)]});
    #
    # subplot(2,1,2)
    # surf(nxy,nyx,abs(E),linestyle = ':',edgecolor='none');
    # xlabel('X')
    # ylabel('Y')
    # colorbar();
    # view(2);
    # axis square;
    ''' цикл по x с дифракцией по y '''
    for idx in range(Nx):
        # for idx in [Nx//2]:
        Fy = np.zeros(Ny, dtype=complex);
        alphay = np.zeros(Ny, dtype=complex);
        betay = np.zeros(Ny, dtype=complex);
        for idy in range(2, Ny - 1):
            Fy[idy] = -E[idx, idy - 1] + 2 * (1 - dy ** 2 / dz / hi) * E[idx, idy] - E[idx, idy + 1];

        Fy[0] = 0;
        Fy[-1] = 0;

        alphay[1] = -Cy[0] / By[0];
        betay[1] = Fy[0] / By[0];

        # for idy in
        #     alphay(idy+1)= -Cy(idy)/(Ay(idy)*alphay(idy) + By(idy));
        #     betay(idy+1) = alphay(idy+1)/Cy(idy)*(Ay(idy)*betay(idy) - Fy(idy));
        '''решение по теории:'''
        for i in range(1, Ny - 1):
            #     # alphay[i+1] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i])
            #     # betay[i+1] = (Fy[i-1]-Ay[i]*betay[i])/(Ay[i]*alphay[i]+By[i])
            alphay[i + 1] = -Cy[i] / (Ay[i] * alphay[i] + By[i]);
            betay[i + 1] = alphay[i + 1] / Cy[i] * (Ay[i] * betay[i] - Fy[i]);
        '''решение из прошлого семестра'''
        # for i in range(2,Ny):
        #     alphay[i] = -Cy[i]/By[i] * 1/(1+Ay[i]/By[i]*alphay[i-1])
        #     betay[i] = (Fy[i-1]-Ay[i]*betay[i-1])/(Ay[i]*alphay[i-1]+By[i])

        E[idx, -1] = (Fy[-1] - Ay[-1] * betay[-1]) / (Ay[-1] * alphay[-1] + By[-1]);
        '''по теории'''
        # for idy in range(Ny-1,0,-1):
        #     E[idx,idy-1] = alphay[idy-1] * E[idx,idy] + betay[idy-1];
        '''прошлый семестр'''
        for idy in range(Ny - 2, -1, -1):
            E[idx, idy] = alphay[idy + 1] * E[idx, idy + 1] + betay[idy + 1]

            # for i in range(1,Nx):
            #     alphax[i+1] = -Cx[i]/Bx[i] * 1/(1+Ax[i]/Bx[i]*alphax[i])
            #     betax[i+1] = (Fx[i]-Ax[i]*betax[i])/(Ax[i]*alphax[i]+Bx[i])
            #
            #
            # E[-1,idy] = (Fx[-1] - Ax[-1]*betax[-1])/(Ax[-1]*alphax[-1] + Bx[-1]);
            # for idx in range(Nx-1,-1,-1):
            #     E[idx-1,idy] = alphax[idx] * E[idx,idy] + betax[idx]


    Ep = np.abs(E) ** 2;

    fig1 = plt.figure(1, figsize=(9.5, 10));
    fig1.clf()
    ax23 = fig1.add_subplot(2, 1, 1, projection='3d')
    ax23.plot_surface(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
    plt.xlabel('X')
    plt.ylabel('Y')
    # plt.colorbar();
    # axis square;
    plt.title(f'z={z:0.2f}\n f={f}');

    ax24 = fig1.add_subplot(2,1, 2)
    ims22 = ax24.pcolormesh(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
    ax24.set_aspect('equal', adjustable='box')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(ims22);
    plt.pause(0.1)
    # print(f'z={z}')

    if idz == Nz-1:

        fig1 = plt.figure(1, figsize=(9.5, 10));
        fig1.clf()

        # ax13 = fig1.add_subplot(2, 2, 1, projection='3d')
        # ax13.plot_surface(nxy, nyx, Ep0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        # plt.xlabel('X')
        # plt.ylabel('Y')
        # # plt.colorbar();
        # # axis square;
        # plt.title(f'z={z:0.2f}\n f={f}');
        #
        # ax14 = fig1.add_subplot(2, 2, 3)
        # ims12 = ax14.pcolormesh(nxy, nyx, Ep0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        # ax14.set_aspect('equal', adjustable='box')
        # plt.xlabel('X')
        # plt.ylabel('Y')
        # plt.colorbar(ims12);

        ax23 = fig1.add_subplot(2, 1, 1, projection='3d')
        ax23.plot_surface(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        plt.xlabel('X')
        plt.ylabel('Y')
        # plt.colorbar();
        # axis square;
        plt.title(f'z={z:0.2f}\n f={f}');

        ax24 = fig1.add_subplot(2, 1, 2)
        ims22 = ax24.pcolormesh(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        ax24.set_aspect('equal', adjustable='box')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.colorbar(ims22);
        # plt.view(2);
        # axis square;
        filename = f'z={z:0.2f}'
        fig1.savefig(f'{direction01}{direction1}{direction2}{filename}.png')
        # plt.pause(0.3)
        #
        # if z == float(f):
        #     fig3 = plt.figure(3, figsize=(12, 14))
        #     fig3.clf()
        #     ax31 = fig3.add_subplot(3, 1, 1)
        #     plt.title(f'Сравнение решений \n z:{z} = f:{f} for x = {x[Nx // 2]}')
        #     point = 1
        #     ax31.scatter(y[::point], Ep[Nx // 2, :][::point], c='r', label='Численной схемой')
        #     ax31.plot(y, Eanprint[Nx // 2, :], label='Аналитически')
        #     plt.legend()
        #     plt.xlabel('Y')
        #     plt.ylabel('I')
        #
        #     ax32 = fig3.add_subplot(3, 2, 3)
        #     plt.title(f'Абсолютная ошибка при x={x[Nx // 2]}')
        #     ax32.stem(y, abs(Eanprint[Nx // 2, :] - Ep[Nx // 2, :]))
        #     plt.xlabel('Y')
        #     plt.ylabel('I')
        #     filename = f'y=0 in z={f}'
        #
        #     ax32 = fig3.add_subplot(3, 2, 4)
        #     plt.title(f'Абсолютная ошибка при y={y[Ny // 2]}')
        #     ax32.stem(x, abs(Eanprint[:, Ny // 2] - Ep[:, Ny // 2]))
        #     plt.xlabel('X')
        #     plt.ylabel('I')
        #     filename = f'x=0 in z={f}'
        #
        #     # fig4 = plt.figure(4,figsize=(9.5,10))
        #     # fig4.clf()
        #     ax41 = fig3.add_subplot(3, 2, 5, projection='3d')
        #     ax41.plot_surface(nxy, nyx, abs(Eanprint - Ep), linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        #     plt.xlabel('X')
        #     plt.ylabel('Y')
        #     # plt.colorbar();
        #     # axis square;
        #     plt.title(f'Абсолютная ошибка при z:{z}=f:{f}');
        #
        #     ax42 = fig3.add_subplot(3, 2, 6)
        #     ims41 = ax42.pcolormesh(nxy, nyx, abs(Eanprint - Ep), linestyle=':', edgecolor='none', cmap=cm.coolwarm);
        #     ax42.set_aspect('equal', adjustable='box')
        #     plt.xlabel('X')
        #     plt.ylabel('Y')
        #     plt.colorbar(ims41);
        #     plt.title(f'Абсолютная ошибка при z:{z}=f:{f}\n'
        #               f'Emax={np.max(abs(Eanprint - Ep)):0.5f}');
        #     plt.tight_layout()
        #
        #     Path(f"{direction01}{direction1}f=z").mkdir(parents=True, exist_ok=True)
        #     fig3.savefig(f'{direction01}{direction1}f=z/{filename}.svg')
        #     fig3.savefig(f'{direction01}{direction1}f=z/{filename}.png')
        #     # Path(f"{direction01}{direction1}f=z").mkdir(parents=True, exist_ok=True)
        #     # filename = f'f=z={z:0.2f}'
        #     #
        #     # fig4.savefig(f'{direction01}{direction1}f=z/{filename}.png')

C:\Users\dimon\AppData\Local\Temp/ipykernel_22236/3872869492.py:119: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims22 = ax24.pcolormesh(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
C:\Users\dimon\AppData\Local\Temp/ipykernel_22236/3872869492.py:156: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims22 = ax24.pcolormesh(nxy, nyx, Ep, linestyle=':', edgecolor='none', cmap=cm.coolwarm);


In [53]:
# Crank2(256,256,50,0.5)

In [41]:
Ep0 = (fft.fftshift(fft.fft2(fft.fftshift(E_s),workers=7)))

dkx = dky = 2*np.pi/(2*X)
kx = ky = np.linspace(-Nx/2,Nx/2,Nx) * dkx
ky,kx = np.meshgrid(kx,ky)
Ef = np.array(None,dtype=complex)
Ef = np.exp(dz*1j*(kx**2 + ky**2)/2) + np.exp(1j*(kx**2+ky**2)/2/float(f))
Ep = Ep0 * Ef
# Ep = Ep0
Ef0 = np.abs(Ep)

In [42]:

fig1 = plt.figure(8, figsize=(9.5, 10));
fig1.clf()
ax13 = fig1.add_subplot(2, 2, 1, projection='3d')
ax13.plot_surface(nxy, nyx, Ef0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
plt.xlabel('X')
plt.ylabel('Y')
# plt.colorbar();
# axis square;
plt.title(f'z={z:0.2f}\n f={f}');

ax14 = fig1.add_subplot(2, 2, 3)
ims12 = ax14.pcolormesh(nxy, nyx, Ef0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);
ax14.set_aspect('equal', adjustable='box')
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(ims12);


C:\Users\dimon\AppData\Local\Temp/ipykernel_22236/3973383026.py:12: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  ims12 = ax14.pcolormesh(nxy, nyx, Ef0, linestyle=':', edgecolor='none', cmap=cm.coolwarm);


In [21]:
np.abs(10+10j)

14.142135623730951

In [40]:
plt.figure()
plt.imshow(np.abs(Ef))